In [ ]:
import gym
from surface_seg.envs.mcs_env import MCSEnv
import gym.wrappers
import numpy as np
import tensorforce 
import copy

In [ ]:

def setup_env(recording=True):
    
    Gs = {}
    Gs["G2_etas"] = np.logspace(np.log10(0.05), np.log10(5.0), num=4)
    Gs["G2_rs_s"] = [0] * 4
    Gs["G4_etas"] = [0.005]
    Gs["G4_zetas"] = [1.0]
    Gs["G4_gammas"] = [+1.0, -1]
    Gs["cutoff"] = 6.5

    G = copy.deepcopy(Gs)

    # order descriptors for simple_nn
    cutoff = G["cutoff"]
    G["G2_etas"] = [a / cutoff**2 for a in G["G2_etas"]]
    G["G4_etas"] = [a / cutoff**2 for a in G["G4_etas"]]
    descriptors = (
        G["G2_etas"],
        G["G2_rs_s"],
        G["G4_etas"],
        G["cutoff"],
        G["G4_zetas"],
        G["G4_gammas"],
    )

    # Set up gym
    MCS_gym = MCSEnv(fingerprints=True, 
                     descriptors = descriptors,
                    permute_seed=None)
    
    if recording:
    # Wrap the gym to provide video rendering every 50 steps
        MCS_gym = gym.wrappers.Monitor(MCS_gym, 
                                         "./vid", 
                                         force=True,
                                        video_callable = lambda episode_id: (episode_id)%50==0)
    
    #Convert gym to tensorfce environment
    env = tensorforce.environments.OpenAIGym(MCS_gym,
                                         max_episode_timesteps=400,
                                         visualize=False)
    
    return env

In [ ]:
recording_env = setup_env()
recording_env.reset()
action = recording_env.environment.action_space.sample() # Random sampling
recording_env.execute(action) # Prints (dict[state], Termination[Bool], observed reward[float]

In [ ]:
from tensorforce.agents import Agent

agent = Agent.create(
    agent='trpo', 
    environment=setup_env(), 
    batch_size=10, 
    learning_rate=1e-2,
    memory = 40000,
    max_episode_timesteps = 400,
    exploration=dict(
        type='decaying', unit='timesteps', decay='exponential',
        initial_value=0.3, decay_steps=1000, decay_rate=0.5
    ))

agent_spec = agent.spec

In [ ]:
from tensorforce.execution import Runner

runner = Runner(
    agent=agent_spec,
    environment=setup_env(),
    max_episode_timesteps=400,
)

%prun runner.run(num_episodes=5)

runner.close()